In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

### Preprocess the "documents" in openbook.txt:

In [3]:
with open("data\Main\openbook.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()
documents = [line.strip() for line in lines]

### Creating Embedding model and using it on OpenBookQA "documents":
- This works as the context retrieval

In [5]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

doc_embeddings = embedder.encode(documents, convert_to_numpy=True)

dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings))

In [15]:
# Query embedding
query = "The sun is responsible for"
query_embedding = embedder.encode([query])

# Search FAISS index
D, I = index.search(np.array(query_embedding), k=1)
print("Best match:", documents[I[0][0]])

Best match: "the sun is a source of heat called sunlight"


In [38]:
# Function to retrieve similar documents
def retrieve_context(query, k=1):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)  # Retrieve top-k similar docs
    return [documents[i] for i in indices[0]]

In [41]:
retrieve_context(query, k=6)

['"the sun is a source of heat called sunlight"',
 '"sunlight produces heat"',
 '"the sun is a source of light called sunlight"',
 '"the sun is the source of solar energy called sunlight"',
 '"sunlight contains ultraviolet light"',
 '"the sun transfers solar energy from itself to the Earth through sunlight"']